# Bugs status. Fuel and MOS.

## Disclaimer
Guys, I'm sorry. We are using inconvenient non-configurable bugtracker for our projects. And we use it for big project with many teams. The only way to make it manageable is by defining extra rules. We are working on simplification of this workflow. But now it this report is the main source of truth for bugs statistics.

The report is based on csv dump that is updated every hour.
## Initialization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

report_columns = ['web_link', 'bug_type', 'severity', 'title', 'tags', 'team', 'assignee', 'status']
milestone = '8.0'
jenkins_host = 'http://172.18.184.144:8080/'
detailed_mos = False

report_from_date = datetime.today().date() - timedelta(days=7)
report_from = str(report_from_date)
print("The report shows changes since " + report_from_date.strftime("week #%U, %a, %b %d") + " " + report_from)

%matplotlib inline

In [ ]:
import urllib2
import json

def get_artifact_url(job_name, artifact_name):
    artifacts_url = jenkins_host + 'job/%s/lastSuccessfulBuild/api/json' % job_name
    r = urllib2.urlopen(artifacts_url)
    artifacts_list = json.loads(r.read())['artifacts']
    artifact_data = filter(lambda x: x['relativePath'].find(artifact_name) != -1, artifacts_list)[0]
    return jenkins_host + 'job/%s/lastSuccessfulBuild/artifact/%s' % (job_name, artifact_data['relativePath'])

print "List of sources for the report"
# These dumps are regenerated twice a day
print(get_artifact_url('lpgrabber-all-%s' % milestone, 'bugs-fuel'))
print(get_artifact_url('lpgrabber-all-%s' % milestone, 'bugs-mos'))
# These delta dumps are regenerated every hour
print(get_artifact_url('lpgrabber-delta-%s' % milestone, 'bugs-fuel'))
print(get_artifact_url('lpgrabber-delta-%s' % milestone, 'bugs-mos'))
print(get_artifact_url('lpgrabber-delta-%s' % milestone, 'teams'))

report_date = get_artifact_url('lpgrabber-delta-%s' % milestone, 'bugs-fuel')[-14:-4]
print "Last update date is", report_date

In [ ]:
# Download csv dumps
def populate_df(url, name):
    csv_file = urllib2.urlopen(url)
    temp_df = pd.DataFrame.from_csv(csv_file)
    print "%s dump size: %d records" % (name, len(temp_df))
    return temp_df

df_fuel_raw = populate_df(get_artifact_url('lpgrabber-all-%s' % milestone, 'bugs-fuel'), 'Fuel')
df_fuel_delta_raw = populate_df(get_artifact_url('lpgrabber-delta-%s' % milestone, 'bugs-fuel'), 'Fuel delta')
df_mos_raw = populate_df(get_artifact_url('lpgrabber-all-%s' % milestone, 'bugs-mos'), 'MOS')
df_mos_delta_raw = populate_df(get_artifact_url('lpgrabber-delta-%s' % milestone, 'bugs-mos'), 'MOS delta')
df_teams_raw = populate_df(get_artifact_url('lpgrabber-delta-%s' % milestone, 'teams'), 'Teams')

## Calculating bugs properties

In [ ]:
[df_fuel, df_fuel_delta, df_mos, df_mos_delta, df_teams] = [df_fuel_raw, df_fuel_delta_raw, df_mos_raw, df_mos_delta_raw, df_teams_raw]
# Add delta data for df_fuel
cols_to_use = df_fuel_delta.columns.difference(df_fuel.columns)
df_fuel = pd.merge(df_fuel, df_fuel_delta[cols_to_use], left_index=True, right_index=True, how='outer')
df_fuel.update(df_fuel_delta.drop_duplicates(keep='last'))
# Add delta data for df_mos
cols_to_use = df_mos_delta.columns.difference(df_mos.columns)
df_mos = pd.merge(df_mos, df_mos_delta[cols_to_use], left_index=True, right_index=True, how='outer')
df_mos.update(df_mos_delta.drop_duplicates(keep='last'))

cols_to_use = df_mos.columns.difference(df_fuel.columns)
df = pd.merge(df_fuel, df_mos[cols_to_use], left_index=True, right_index=True, how='outer')
df.update(df_mos.drop_duplicates(keep='last'))


# Membership in these teams doesn't related to any area. We don't track membership for them.
garbage_teams = [
    'mos-security', 'fuel-security', 'fuel-bugs', 'fuel-dev', 'fuel-drivers', 'mos-drivers',
    'fuel-core', 'mos-bugs', 'mos-da', 'fuel', 'mos-all', 'fuel-bugfix-team', 'fuel-enhancements-team',
    'fuel-all', 'fuel-plugins-drivers', 'fuel-plugins-bugs', 'fuel-mixed-team', 'mos-platform-components',
    'fuel-ui-team', 'fuel-telco-team', 'fuel-octane-team', 'fuel-octane-core', 'fuel-partner-eng-dev',
    'fuel-partner-core', 'fuel-plugin-external-glusterfs', 'fuel-noop-fixtures-drivers'
]
df_teams.drop(garbage_teams, inplace=True, errors='ignore', axis=1)

# Get list of teams for every person
dftt = pd.Series(df_teams.apply(lambda r: ' '.join([i for i in r if isinstance(i, str)]), axis=1), name='team')
for t in df_teams.columns:
    dftt.loc[t] = t

fuel_milestones = sorted([x for x in df.columns if x.endswith('importance') & x.startswith('fuel_')], reverse=True)
mos_milestones = sorted([x for x in df.columns if x.endswith('importance') & x.startswith('mos_')], reverse=True)

def get_milestone(bug):
    for m in fuel_milestones:
        if pd.notnull(bug[m]):
            return m[:-11]
    for m in mos_milestones:
        if pd.notnull(bug[m]):
            return m[:-11]
    return np.nan

def get_ms_param(bug, param):
    return bug['%s_%s' % (bug['milestone'], param)]


def get_bug_close_date(bug, prj='fuel'):
    field_prefix = bug['milestone']
    # This is an easiest but buggy way to get date of marking bug as a duplicate.
    # It is precise enough for us.
    if pd.notnull(bug.duplicates):
        return bug['date_last_updated']
    status = bug[field_prefix + '_status']
    if status == 'Fix Committed':
        return bug[field_prefix + '_date_fix_committed']
    if status == 'Fix Released':
        if bug[field_prefix + '_date_fix_committed'] != 'None':
            return bug[field_prefix + '_date_fix_committed']
        return bug[field_prefix + '_date_fix_released']
    if status == 'Incomplete':
        return bug[field_prefix + '_date_incomplete']
    if status in ['Opinion', 'Invalid', "Won't Fix", 'Expired']:
        return bug[field_prefix + '_date_closed']
    return float('nan')

def get_bug_type(bug):
    if (
        bug.area in ['area-python', 'area-library', 'area-ui', 'no-area', 'area-mos'] or
        bug.area.startswith('area-mos-') or
        (bug.area == 'area-qa' and 'ostf' in bug.tags) or
        (bug.area == 'area-partners' and 'vcenter' in bug.tags)
    ):
        if 'fuel-devops' in bug.tags:
            return 'tools'
        if 'fuel-dev-tools' in bug.tags:
            return 'tools'
        # This is a corner case. In rare case stats bug can affect user. About half of stats bugs don't affect ISO
        # We've decided to count all of the stats bugs as our internal tool issues.
        if 'feature-stats' in bug.tags:
            return 'tools'
        if 'feature' in bug.tags:
            return 'feature'
        # 'covered-by-bp' and 'need-bp' are lists of bugs and features tracked as bugs
        # that we aren't going to fix without related blueprints implementation.
        # Frankly, we should review these lists on regular basis.
        if 'covered-by-bp' in bug.tags:
            return 'bp'
        if 'need-bp' in bug.tags:
            return 'bp'
        if 'docs' in bug.tags:
            return 'doc'
        if 'tech-debt' in bug.tags:
            return 'tech debt'
        return 'bug'
    # We should have separate type for this someday. But now we don't care
    if bug.area in ['area-partners', 'area-plugins']:
        return 'tools'
    if bug.area == 'area-docs' or 'docs' in bug.tags:
        return 'doc'
    # We have internal bugs, features, tech-debt and tools development.
    # But noone is interested in statistics of this kind right now.
    return 'task'

def get_bug_severity(importance):
    if importance in ['Critical', 'High']:
        return 'high'
    if importance in ['Medium', 'Low', 'Wishlist']:
        return 'low'
    return 'untriaged'

def get_bug_is_regression(bug):
    # In general we don't know if the bug is a regression or not.
    # Right now we consider every open bug with bug_type=='bug' as
    # a bug in new feature. Until it has 'late-discovery' or 'regression-8.0' tag.
    # For closed bugs we don't triage this kind of information.
    if 'late-discovery' in bug.tags and 'regression-8.0' in bug.tags:
        return '?'
    if set([
        'regression-8.0', 'module-octane', 'ubuntu-bootstrap', 'feature-nodegroup',
        'feature-network-template', 'feature-reduced-footprint',
        'feature-openstack-config-change', 'team-linux', 'team-ubuntu', 'feature-centos7'
        ]).intersection(bug.tags):
        return True
    if set(['late-discovery', 'customer-found', 'support']).intersection(bug.tags):
        return False
    if bug['fuel_7.0_status'] == 'Invalid':
        return True
    if bug['fuel_7.0_status'] in ['Fix Committed', 'Fix Released', 'New', 'Confirmed', 'In Progress', "Won't Fix"]:
        return False
    if bug.date_created <= "2015-09-29":
        return False
    if bug.date_created >= "2015-12-03":
        return True
    return False

def get_bug_short_status(bug):
    if pd.notnull(bug['duplicates']):
        return 'rejected'
    if pd.notnull(bug['fuel_%s_status' % milestone]):
        s = bug['fuel_%s_status' % milestone]
    elif pd.notnull(bug['mos_%s_status' % milestone]):
        s = bug['mos_%s_status' % milestone]
    else:
        return 'not_for_%s' % milestone
        s = bug['status']
    if s in ['New', 'Confirmed', 'Triaged', 'In Progress']:
        return 'open'
    if s in ['Incomplete', 'Incomplete (with response)', 'Incomplete (without response)']:
        return 'rejected'
        return 'incomplete' # or rejected in case if we don't need to separate this area
    if s in ['Opinion', 'Invalid', "Won't Fix", 'Expired']:
        return 'rejected'
    if s in ['Fix Committed', 'Fix Released']:
        return 'fixed'
    return 'unknown'

def get_fuel_area(bug):
    for team in ['ci', 'build', 'ui', 'library', 'python', 'devops', 'qa', 'docs']:
        if bug.team == 'fuel-' + team:
            return 'area-' + team
    if bug.team == 'fuel-partner':
        return 'area-partners'
    if bug.team == 'fuel-octane':
        return 'area-python'
    for tag in ['ui', 'docs']:
        if tag in bug.tags:
            return 'area-' + tag
    return None

    
def get_mos_area(bug):
    if not detailed_mos:
        fuel_area = get_fuel_area(bug)
        if fuel_area:
            return fuel_area
        return 'area-mos'
    for team in [
        'mos-neutron', 'mos-ceilometer', 'mos-ceph', 'mos-cinder', 'mos-glance', 'mos-heat', 'mos-horizon',
        'mos-ironic', 'mos-keystone', 'mos-kernel-virt', 'mos-kernel-networking', 'mos-kernel-storage',
        'mos-linux', 'mos-murano', 'mos-nova', 'mos-oslo', 'mos-packaging', 'mos-puppet', 'mos-sahara', 'mos-scale',
        'mos-swift', 'mos-maintenance']:
        if bug.team == team:
            return 'area-' + team
    if bug.team == 'mos-qa':
        return 'area-qa'
    for tag in ['horizon', 'cinder', 'neutron', 'sahara', 'ironic', 'keystone', 'neutron', 'murano', 'swift']:
        if tag in bug.tags:
            return 'area-mos-' + tag
    fuel_area = get_fuel_area(bug)
    if fuel_area:
        return fuel_area
    return 'no-area'

def get_bug_area(bug, guess_empty=False):
    if filter(lambda x: x.startswith('area-'), bug.tags):
        if ' '.join(filter(lambda x: x.startswith('area-'), bug.tags)) in ['area-mos', 'area-linux']:
            return get_mos_area(bug)
        if type(bug.assignee) == str and bug.assignee.startswith('fuel-'):
            area_suffix = filter(lambda x: x.startswith('area-'), bug.tags)[0][5:]
            assignee_suffix = bug.assignee[5:]
            if area_suffix == 'python' and assignee_suffix == 'octane':
                return 'area-python'
            if area_suffix == 'partners' and assignee_suffix.startswith('partner'):
                return 'area-partners'
            if area_suffix == 'plugins' and assignee_suffix.startswith('plugin'):
                return 'area-plugins'
            # We don't track bugs movement between areas. It needs to be supported in lp-reports tool.
            #if area_suffix != assignee_suffix:
            #    return 'changed-area'
        if len(filter(lambda x: x.startswith('area-'), bug.tags)) > 1:
            return 'mixed-areas'
        return ' '.join(filter(lambda x: x.startswith('area-'), bug.tags))
    if guess_empty:
        return get_mos_area(bug)
    return 'no-area'

df['web_link'] = ["https://launchpad.net/bugs/%s" % x for x in df.index]
df['tags'] = [x[1:-1].split(', ') for x in df.tags]
df['milestone'] = df.apply(get_milestone, axis=1)
df['status'] = df.apply(get_ms_param, axis=1, param='status')
df['importance'] = df.apply(get_ms_param, axis=1, param='importance')
df['assignee'] = df.apply(get_ms_param, axis=1, param='assignee')
df['short_status'] = df.apply(get_bug_short_status, axis=1)
df = df.join(dftt, on='assignee')
df['date_closed'] = df.apply(get_bug_close_date, axis=1)
df['area'] = df.apply(get_bug_area, guess_empty=True, axis=1)
df['bug_type'] = df.apply(get_bug_type, axis=1)
df['severity'] = [get_bug_severity(x) for x in df['importance']]
df['is_regression'] = df.apply(get_bug_is_regression, axis=1)
df['delivery_team'] = [' '.join(filter(lambda x: x.startswith('team-'), tags)) for tags in df['tags']]
df['swarm_blocker'] = ['swarm-blocker' in x for x in df.tags]
df['customer_found'] = [('customer-found' in x) or ('support' in x) for x in df.tags]
df['long_haul_testing'] = ['long-haul-testing' in x for x in df.tags]

print "Bugs by area in both projects: %d" % len(df)
df.groupby(['area', 'short_status']).size().unstack().fillna(0)

In [ ]:
df.pivot_table('title', index=['area', 'bug_type'], columns=['importance', 'short_status'], aggfunc=len, fill_value=0, margins=True)

## Bugs in unknown areas

In [ ]:
q = "area=='no-area'"
l = len(df.query(q))
print "Bugs in unknown area: %d" % l
if l:
    df.query(q)[['team', 'assignee']].groupby(['team', 'assignee']).size()

### Unknown areas bugs list
#### Bugs targeted to several areas

In [ ]:
c = ['web_link', 'team', 'tags', 'assignee']
df.query("area=='mixed-areas'")[c]

#### Bugs without areas

In [ ]:
df[df.duplicates.isnull()].query("area=='no-area'")[c]

## Community report

In [ ]:
from jinja2 import Template

def numbers_by_areas(query):
    total = len(df.query(query + "& short_status=='open'" + "& area in ['area-library', 'area-python', 'area-ui']"))
    ui = len(df.query(query + "& short_status=='open'" + "& area=='area-ui'"))
    python = len(df.query(query + "& short_status=='open'" + "& area=='area-python'"))
    library = len(df.query(query + "& short_status=='open'" + "& area=='area-library'"))
    opened_total = len(df.query(query + "& date_created >= @report_from" + "& area in ['area-library', 'area-python', 'area-ui']"))
    opened_ui = len(df.query(query + "& date_created >= @report_from" + "& area=='area-ui'"))
    opened_python = len(df.query(query + "& date_created >= @report_from" + "& area=='area-python'"))
    opened_library = len(df.query(query + "& date_created >= @report_from" + "& area=='area-library'"))
    closed_total = len(df.query(query + "& date_closed >= @report_from" + "& area in ['area-library', 'area-python', 'area-ui']"))
    closed_ui = len(df.query(query + "& date_closed >= @report_from" + "& area=='area-ui'"))
    closed_python = len(df.query(query + "& date_closed >= @report_from" + "& area=='area-python'"))
    closed_library = len(df.query(query + "& date_closed >= @report_from" + "& area=='area-library'"))
    return "%d(%d/%d/%d) +%d(%d/%d/%d) -%d(%d/%d/%d)" % (
        total, ui, python, library, opened_total, opened_ui, opened_python, opened_library, closed_total,
        closed_ui, closed_python, closed_library)

def numbers_without_areas(query):
    total = len(df.query(query + "& short_status=='open'"))
    total_high = len(df.query(query + "& short_status=='open'" + "& severity=='high'"))
    opened_total = len(df.query(query + "& date_created >= @report_from"))
    opened_high = len(df.query(query + "& date_created >= @report_from" + "& severity=='high'"))
    closed_total = len(df.query(query + "& date_closed >= @report_from"))
    closed_high = len(df.query(query + "& date_closed >= @report_from" + "& severity=='high'"))
    return "%d(%d) +%d(%d) -%d(%d)" % (
        total, total_high, opened_total, opened_high, closed_total, closed_high)

def numbers_for_management(query):
    total = len(df.query(query + "& short_status=='open'"))
    opened_total = len(df.query(query + "& date_created >= @report_from & short_status != 'rejected'"))
    closed_total = len(df.query(query + "& date_closed >= @report_from & short_status != 'rejected'"))
    return "%d (+%d,-%d)" % (total, opened_total, closed_total)


t_public = Template("""
Report needs to be copied here: https://etherpad.openstack.org/p/fuel-bugs-status

Report date: {{report_date}}. Numbers show difference since {{report_from}}

Bugs in UI, python, library

Format: Total open(UI open/Python open/Library open) +Total income (UI/Python/Library) -Total outcome(UI/Python/Library)

Real bugs (defects):
- Critical/high: {{bugs_high}}
- Medium/low: {{bugs_low}}
Features tracked as bug reports:
- Critical/high: {{bugs_features_high}}
- Medium/low: {{bugs_features_low}}
Technical debt bugs:
- Critical/high: {{bugs_tech_debt_high}}
- Medium/low: {{bugs_tech_debt_low}}

High priority defects on delivery teams:
{% for t in bugs_delivery_teams|sort %}- {{t}}: {{bugs_delivery_teams[t]}}
{% endfor %}
Bugs in other teams. Format: Total open(High) +Total income(High) -Total outcome(High)
- QA: {{bugs_qa}}
- Docs: {{bugs_docs}}
- Devops: {{bugs_devops}}
- Build: {{bugs_build}}
- CI: {{bugs_ci}}
- Infra Apps: {{bugs_apps}}
- MOS: {{bugs_mos}}
- Partners: {{bugs_partners}}
- Plugins: {{bugs_plugins}}

Extra filters:
- Swarm blockers: {{bugs_swarm}}
- Customer found: {{bugs_cf}}
- Security bugs: {{bugs_security}}
- Long haul testing: {{bugs_longhaul}}
- Affects 7.0: {{bugs_late_discovery}}
- Regressions: {{bugs_regressions}}
- QA/Docs/Infra: {{bugs_qa_docs_infra}}

{{noarea}} bugs don't have or have misleading area tag and not included into this report:
{% for item in noarea_bugs %}{{item}}
{% endfor %}
""")

delivery_teams_report = {}
for t in df['delivery_team'].value_counts().index:
    delivery_teams_report[t] = numbers_without_areas("delivery_team=='%s' & severity=='high' & bug_type=='bug' & area in ['area-python', 'area-library']" % t)

delivery_teams_report = {k:v for k, v in delivery_teams_report.items() if v != '0(0) +0(0) -0(0)'}

if '' in delivery_teams_report:
    delivery_teams_report['no-team'] = delivery_teams_report['']
    del delivery_teams_report['']
    

public_report = t_public.render(
    report_from=report_from,
    report_date=report_date,
    bugs_high=numbers_by_areas("severity=='high' & bug_type=='bug'"),
    bugs_low=numbers_by_areas("severity=='low' & bug_type=='bug'"),
    bugs_features_high=numbers_by_areas("severity=='high' & bug_type in ['feature', 'bp']"),
    bugs_features_low=numbers_by_areas("severity=='low' & bug_type in ['feature', 'bp']"),
    bugs_tech_debt_high=numbers_by_areas("severity=='high' & bug_type=='tech debt'"),
    bugs_tech_debt_low=numbers_by_areas("severity=='low' & bug_type=='tech debt'"),
    bugs_qa=numbers_without_areas("area=='area-qa'"),
    bugs_docs=numbers_without_areas("area=='area-docs'"),
    bugs_devops=numbers_without_areas("area=='area-devops'"),
    bugs_build=numbers_without_areas("area=='area-build'"),
    bugs_ci=numbers_without_areas("area=='area-ci'"),
    bugs_apps=numbers_without_areas("area=='area-infra-apps'"),
    bugs_mos=numbers_without_areas("area>='area-mos' & area<='area-mot'"),
    bugs_partners=numbers_without_areas("area=='area-partners'"),
    bugs_plugins=numbers_without_areas("area=='area-plugins'"),
    bugs_swarm=numbers_by_areas("swarm_blocker==True"),
    bugs_cf=numbers_by_areas("customer_found==True & bug_type=='bug'"),
    bugs_security=numbers_by_areas("security_related==1 & bug_type=='bug'"),
    bugs_longhaul=numbers_by_areas("long_haul_testing==True & bug_type=='bug'"),
    bugs_late_discovery=numbers_by_areas("severity=='high' & bug_type=='bug' & is_regression==False"),
    bugs_regressions=numbers_by_areas("severity=='high' & bug_type=='bug' & is_regression==True"),
    bugs_qa_docs_infra=numbers_without_areas("area in ['area-qa', 'area-docs', 'area-devops', 'area-ci', 'area-build']"),
    bugs_delivery_teams=delivery_teams_report,
    bugs_delivery_bugfix=numbers_without_areas("delivery_team=='team-bugfix' & severity=='high' & area=='area-python' & bug_type=='bug'"),
    bugs_delivery_network=numbers_without_areas("delivery_team=='team-network' & severity=='high' & area=='area-python' & bug_type=='bug'"),
    bugs_delivery_enhancements=numbers_without_areas("delivery_team=='team-enhancements' & severity=='high' & area=='area-python' & bug_type=='bug'"),
    bugs_delivery_upgrades=numbers_without_areas("delivery_team=='team-upgrades' & severity=='high' & area=='area-python' & bug_type=='bug'"),
    bugs_delivery_ironic=numbers_without_areas("delivery_team=='team-ironic' & severity=='high' & area=='area-python' & bug_type=='bug'"),
    bugs_delivery_linux=numbers_without_areas("delivery_team=='team-linux' & severity=='high' & area=='area-python' & bug_type=='bug'"),
    bugs_delivery_ubuntu=numbers_without_areas("delivery_team=='team-ubuntu' & severity=='high' & area=='area-python' & bug_type=='bug'"),
    bugs_delivery_noteam=numbers_without_areas("delivery_team=='' & severity=='high' & area=='area-python' & bug_type=='bug'"),
    noarea=len(df.query("area in ['no-area', 'changed-team', 'mixed-areas']")),
    noarea_bugs=df.query("area in ['no-area', 'changed-team', 'mixed-areas']")['web_link']
    )
print public_report

## Management report

In [ ]:
from IPython.core.display import HTML

names = [
    "(high) User-affecting bugs from previous releases",
    "(high) User-affecting bugs for features being implemented in " + milestone,
    "(high) Enhancements",
    "(high) Technical debt",
    "(high) QA, Infra",
    "(high) Docs",
    "(low) User-affecting bugs",
    "(low) Enhancements",
    "(low) Technical debt",
    "(low) QA, Infra",
    "(low) Docs",
    "support & customer-found",
    "long-haul-testing",
    "swarm-blocker",
    "security bugs",    
]
queries = [
    "bug_type=='bug' & severity=='high' & is_regression==False & area in ['area-python', 'area-library', 'area-ui']",
    "bug_type=='bug' & severity=='high' & is_regression==True & area in ['area-python', 'area-library', 'area-ui']",
    "bug_type in ['feature', 'bp'] & severity=='high' & area in ['area-python', 'area-library', 'area-ui']",
    "bug_type=='tech debt' & severity=='high' & area in ['area-python', 'area-library', 'area-ui']",
    "severity=='high' & area in ['area-qa', 'area-devops', 'area-ci', 'area-build']",
    "severity=='high' & area in ['area-docs']",
    "bug_type=='bug' & severity=='low' & area in ['area-python', 'area-library', 'area-ui']",
    "bug_type in ['feature', 'bp'] & severity=='low' & area in ['area-python', 'area-library', 'area-ui']",
    "bug_type=='tech debt' & severity=='low' & area in ['area-python', 'area-library', 'area-ui']",
    "severity=='low' & area in ['area-qa', 'area-devops', 'area-ci', 'area-build']",
    "severity=='low' & area in ['area-docs']",
    "customer_found==True & bug_type=='bug' & area in ['area-python', 'area-library', 'area-ui']",
    "long_haul_testing==True & bug_type=='bug' & area in ['area-python', 'area-library', 'area-ui']",
    "swarm_blocker==True & bug_type=='bug' & area in ['area-python', 'area-library', 'area-ui']",
    "security_related==1 & bug_type=='bug' & area in ['area-python', 'area-library', 'area-ui']",
]
queries = [
    "bug_type=='bug' & area in ['area-python', 'area-library', 'area-ui']",
    "bug_type in ['feature', 'bp'] & area in ['area-python', 'area-library', 'area-ui']",
    "area in ['area-qa', 'area-devops', 'area-ci', 'area-build', 'area-infra-apps']",
    "(area in ['area-docs'] | (area in ['area-python', 'area-library', 'area-ui'] & bug_type=='doc'))",
    "bug_type in ['tech debt', 'tools'] & area in ['area-python', 'area-library', 'area-ui']",
    "area in ['area-python', 'area-library', 'area-ui', 'area-qa', 'area-devops', 'area-ci', 'area-build', 'area-infra-apps', 'area-docs']",
]

output = "<table border><tr><th>Query<th>Critical<th>High<th>Medium<th>Low</tr>"

for q in queries:
    output += "<tr><th>" + q
    for priority in ['Critical', 'High', 'Medium']:
        output += "<td>" + numbers_for_management(q + "& importance==@priority")
    output += "<td>" + numbers_for_management(q + "& importance in ['Low', 'Wishlist']")
    output += "</tr>"

HTML(output)

## General numbers for high priority defects

In [ ]:
df.query("bug_type=='bug' & severity=='high'")['short_status'].value_counts()

In [ ]:
df.query("severity=='high' & short_status=='open'")['bug_type'].value_counts()

In [ ]:
df[df.duplicates.isnull()].query("severity=='high' & short_status=='rejected'")['status'].value_counts()

In [ ]:
len(df[df.duplicates.notnull()].query("severity=='high' & short_status=='rejected'"))

## High priority bugs trends

In [ ]:
def get_date_from_str(date):
    if pd.isnull(date):
        return np.nan
    return pd.to_datetime(date)
    #if date>='2015':
    #    return "w%02d" % pd.to_datetime(date).isocalendar()[1] # N of week
    return date[:7]

q = "bug_type=='bug' & area in ['area-library', 'area-python'] & date_created > '0'"
q = "bug_type=='bug' & severity=='high' & date_created > '0'"
df['month'] = [get_date_from_str(x) for x in df['date_created']]
df['month_closed'] = [get_date_from_str(x) for x in df['date_closed']]
total_created = df.query(q).pivot_table('title', index='month', columns='area', aggfunc=len).fillna(0)
total_closed = df.query(q).pivot_table('title', index='month_closed', columns='area', aggfunc=len).fillna(0)
total_rejected = df.query(q + "& short_status=='rejected'").pivot_table('title', index='month_closed', columns='area', aggfunc=len).fillna(0)
total_fixed = df.query(q + "& short_status=='fixed'").pivot_table('title', index='month_closed', columns='area', aggfunc=len).fillna(0)

In [ ]:
total_created.resample('w', how='sum')[pd.to_datetime('today')-pd.Timedelta('120 days'):].plot(title="Bugs creation", figsize=(15, 5))
total_closed.resample('w', how='sum')[pd.to_datetime('today')-pd.Timedelta('120 days'):].plot(title="Bugs closing", figsize=(15, 5))
total_rejected.resample('w', how='sum')[pd.to_datetime('today')-pd.Timedelta('120 days'):].plot(title="Bugs rejected", figsize=(15, 5))
total_fixed.resample('w', how='sum')[pd.to_datetime('today')-pd.Timedelta('120 days'):].plot(title="Bugs fixed", figsize=(15, 5))